In [1]:
DATA_PATH = "../../data"

In [2]:
import polars as pl

train = (
    pl.read_csv(f"{DATA_PATH}/train.csv")
    .with_columns(
        pl.col("prompt").str.json_decode(),
        pl.col("response_a").str.json_decode(),
        pl.col("response_b").str.json_decode(),
    )
    .with_columns(  # 長さの情報を追加する
        pl.col("prompt")
        .map_elements(lambda x: len(x), return_dtype=pl.Int64)
        .alias("len_prompt"),
        pl.col("response_a")
        .map_elements(lambda x: len(x), return_dtype=pl.Int64)
        .alias("len_response_a"),
        pl.col("response_b")
        .map_elements(lambda x: len(x), return_dtype=pl.Int64)
        .alias("len_response_b"),
    )
    .with_columns(  # 最後のレスポンスのみを取得する
        pl.col("prompt")
        .map_elements(lambda x: x[-1], return_dtype=pl.String)
        .alias("last_prompt"),
        pl.col("response_a")
        .map_elements(lambda x: x[-1], return_dtype=pl.String)
        .alias("last_response_a"),
        pl.col("response_b")
        .map_elements(lambda x: x[-1], return_dtype=pl.String)
        .alias("last_response_b"),
    )
    .with_columns(  # 最後のレスポンスがNoneの場合を空文字にする、約60件程度
        pl.col("last_response_a").fill_null(""),
        pl.col("last_response_b").fill_null(""),
    )
    .with_columns(  # labelを付与する
        pl.when(pl.col("winner_model_a") == 1)
        .then(0)
        .when(pl.col("winner_model_b") == 1)
        .then(1)
        .when(pl.col("winner_tie") == 1)
        .then(2)
        .alias("label"),
    )
    .select(  # 元のprompt, responseを削除する
        pl.exclude(["prompt", "response_a", "response_b"])
    )
)

In [3]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

fold_arr = np.zeros(train.height)
sgkf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for idx, (_, val_idx) in enumerate(
    sgkf.split(train, train["label"])
):
    fold_arr[val_idx] = idx

train = train.with_columns(pl.Series(fold_arr).cast(pl.Int64).alias("fold"))

In [ ]:
# idとlabelをdictにして保存する
id_label_dict = dict(zip(train["id"], train["fold"]))

In [ ]:
# jsonで保存する
import json

with open(f"{DATA_PATH}/label_stratified_fold.json", "w") as f:
    json.dump(id_label_dict, f)

In [ ]:
train["label"].value_counts()

label,count
i32,u32
1,19652
0,20064
2,17761


In [ ]:
train["fold"].value_counts()

fold,count
i64,u32
2,19159
1,19159
0,19159
